<span style="font-width:bold; font-size: 3rem; color:#333;">- Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Parse Data
2. Feature Group Insertion

### <span style='color:#ff5f27'> 📝 Imports

In [1]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import util
import warnings
warnings.filterwarnings("ignore")

### Get AQI API Token

Go here first, to get your AQI API key
https://aqicn.org/data-platform/token/

Then enter the token you copied into the following variable in the code below. 

AQI_API_KEY = 

In [2]:
# Getting the current date
today = datetime.date.today()
yesterday = today - datetime.timedelta(days = 1)

country="sweden"
city="stockholm"
street="stockholm-hornsgatan-108-gata"

project = hopsworks.login()
fs = project.get_feature_store() 

# Get your AQI_API_KEY from here:
# https://aqicn.org/data-platform/token/

AQI_API_KEY = 'bf455e04dd016b4cb3c13e68a994bd9aaea32284'
#AQI_API_KEY = hopsworks.connection().get_secrets_api().get_secret("AQI_API_KEY").value


Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://snurran.hops.works/p/8321
Connected. Call `.close()` to terminate connection gracefully.


### <span style="color:#ff5f27;"> 🔮 Connecting to Hopsworks Feature Store </span>

In [3]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Get Yesterday's Air Quality data (PM2.5) from the Feature Group</span>
My Air quality sensor is:  https://aqicn.org/station/sweden/stockholm-sodermalm/ 

In [4]:
# Read all data from the feature group
df_air_quality = air_quality_fg.read()

df = pd.DataFrame()
df_air_quality['day'] = pd.to_datetime(df_air_quality['date']).dt.date

Finished: Reading data from Hopsworks, using ArrowFlight (0.49s) 


In [5]:
# Extracting the "date" and "city_name" columns from the 'df_air_quality' DataFrame
# Grouping the data by "city_name" and finding the maximum date for each city
last_dates_aq = df_air_quality[["date", "city", "street"]].groupby("street").max()

# Converting the date values to string format for consistency
last_dates_aq.date = last_dates_aq.date.astype(str)

In [6]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(country, city, street, today, AQI_API_KEY)
aq_today_df

,pm25,country,city,street,date
0,51.0,sweden,stockholm,stockholm-hornsgatan-108-gata,2024-03-13


In [7]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
dtypes: datetime64[ns](1), float32(1), object(3)
memory usage: 164.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather data</span>

In [8]:
hourly_df = util.get_hourly_weather_forecast(city)
hourly_df = hourly_df.set_index('date')

## We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 59.19999694824219°N 18.0°E
Elevation 18.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-03-13,3.70,0.0,10.002560,210.256348,stockholm
1,2024-03-14,8.25,0.4,18.398781,210.579147,stockholm
2,2024-03-15,7.30,0.2,16.981165,184.864426,stockholm
3,2024-03-16,7.30,0.3,12.413476,196.858475,stockholm
4,2024-03-17,0.25,0.1,22.608458,350.837708,stockholm
5,2024-03-18,2.25,0.0,12.413477,29.538700,stockholm
6,2024-03-19,3.35,0.0,5.400000,323.130035,stockholm
7,2024-03-20,6.90,0.0,6.830519,161.564957,stockholm
8,2024-03-21,6.00,0.0,10.080000,180.000000,stockholm
9,2024-03-22,6.30,0.0,14.264361,190.175446,stockholm


In [9]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 448.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [10]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2024-03-13 13:41:42,086 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://snurran.hops.works/p/8321/fs/8269/fg/9304


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://snurran.hops.works/p/8321/jobs/named/air_quality_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f37da26e380>,
 {
   "evaluation_parameters": {},
   "meta": {
     "great_expectations_version": "0.15.12",
     "expectation_suite_name": "aq_expectation_suite",
     "run_id": {
       "run_time": "2024-03-13T13:41:42.086799+00:00",
       "run_name": null
     },
     "batch_kwargs": {
       "ge_batch_id": "6cdfd180-e13f-11ee-9c8f-ea877cc8f5de"
     },
     "batch_markers": {},
     "batch_parameters": {},
     "validation_time": "20240313T134142.086754Z",
     "expectation_suite_meta": {
       "great_expectations_version": "0.15.12"
     }
   },
   "results": [
     {
       "result": {
         "observed_value": 51.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-0

In [11]:
# Insert new data
weather_fg.insert(daily_df)

2024-03-13 13:41:44,775 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://snurran.hops.works/p/8321/fs/8269/fg/9305


Uploading Dataframe: 0.00% |          | Rows 0/10 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://snurran.hops.works/p/8321/jobs/named/weather_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f37da4f9f60>,
 {
   "evaluation_parameters": {},
   "meta": {
     "great_expectations_version": "0.15.12",
     "expectation_suite_name": "weather_expectation_suite",
     "run_id": {
       "run_time": "2024-03-13T13:41:44.774804+00:00",
       "run_name": null
     },
     "batch_kwargs": {
       "ge_batch_id": "6e7a036c-e13f-11ee-9c8f-ea877cc8f5de"
     },
     "batch_markers": {},
     "batch_parameters": {},
     "validation_time": "20240313T134144.774755Z",
     "expectation_suite_meta": {
       "great_expectations_version": "0.15.12"
     }
   },
   "results": [
     {
       "result": {
         "observed_value": 5.399999618530273,
         "element_count": 10,
         "missing_count": null,
         "missing_percent": null
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "valid

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
